In [1]:
%%time
import pandas as pd
import os, sys, time, json, re, string

from pyspark import SparkContext, SparkConf, StorageLevel, keyword_only

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.param.shared import HasInputCol, HasInputCols, HasOutputCol, HasOutputCols, Param
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer, NGram, CountVectorizer, StopWordsRemover
from pyspark.ml.feature import VectorAssembler, PCA

from pyspark.ml.classification import LogisticRegression, NaiveBayes, DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, MultilayerPerceptronClassifier
from pyspark.ml import Pipeline, Transformer

from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from platform import python_version
print(python_version())

2.7.14
CPU times: user 68.3 ms, sys: 12.8 ms, total: 81.1 ms
Wall time: 74.5 ms


In [2]:
%%time
sc

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10 µs


<SparkContext master=yarn appName=PySparkShell>

In [3]:
%%time
spark = SparkSession.builder \
        .appName("fakenews_baseline_model_with_pca") \
        .config("spark.master", "yarn") \
        .config("spark.submit.deployMode", "cluster") \
        .config("spark.driver.memory", "15g") \
        .config("spark.executor.instances", "5") \
        .config("spark.executor.cores", "4") \
        .config("spark.executor.memory", "10g") \
        .getOrCreate()

CPU times: user 2.02 ms, sys: 7.84 ms, total: 9.86 ms
Wall time: 14.2 ms


In [4]:
%%time
fakenews_path="gs://dataproc-6ca41800-27b4-47d5-abee-55c011dfa389-asia-southeast1/data/fake-news/"
fakenews_data_path = fakenews_path + "two_million_rows_news_cleaned_2018_02_13_pyspark.csv"
df_news = spark.read.format("com.databricks.spark.csv") \
                    .option("header", "true") \
                    .option("delimiter", '#') \
                    .load(fakenews_data_path)

# only keep type and content
df_news = df_news.select("type", "content", "domain")
# add binary label
df_news = df_news.withColumn("label", F.when(F.col("type") == 'fake', 1).otherwise(0))

#remove empty content which will cause problem when transform the text
df_news = df_news.filter(df_news.content != "")

#facing out of memory issue in google cluster for 5 instance * 15 GB, so sampling with 10K fake news and non-fake news
row_count = 10000
df_news_fake = df_news.filter(df_news.type == 'fake').limit(row_count)
df_news_nonfake = df_news.filter(df_news.type != 'fake').limit(row_count)
df_news = df_news_fake.union(df_news_nonfake)

# split the dataset
df_train, df_test = df_news.randomSplit([0.8, 0.2], seed=666)

param_tuning = False

CPU times: user 13.9 ms, sys: 5.19 ms, total: 19.1 ms
Wall time: 8.88 s


In [5]:
%%time
# customized transformer class to manually extract some counting based text features
class ReviewContentTransformer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol="content", outputCol="content_features"):
        super(ReviewContentTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)


    def _transform(self, dataset):
        @udf('double')
        def f(s):
            uppercase_count = 0
            char_count = 0
            for c in s:                
                if c in string.ascii_uppercase:
                    uppercase_count += 1
                    char_count += 1
                elif c in string.ascii_lowercase:
                    char_count += 1
            
            text_len = len(s)
            return Vectors.dense(text_len, char_count, 
                                 uppercase_count, uppercase_count / (char_count + 1e-10))

        return dataset.withColumn(self.getOutputCol(), 
                                  F.udf(f, VectorUDT())(dataset[self.getInputCol()]))

CPU times: user 157 µs, sys: 30 µs, total: 187 µs
Wall time: 145 µs


In [6]:
%%time
# customized transformer class to manually extract some counting based word features
class ReviewWordsTransformer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol="content", outputCol="content_features"):
        super(ReviewWordsTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)


    def _transform(self, dataset):
        
        def f(words):    
            word_count = len(words)
            unique_word_count = len(set(words))
            upper_words = []
            for w in words:
                if w.isupper():
                    upper_words.append(w)
            upper_word_count = len(set(upper_words))
            unique_upper_word_count = len(upper_words)
            return Vectors.dense(word_count, unique_word_count, unique_word_count / (word_count + 1e-10),
                                 upper_word_count, upper_word_count / (word_count + 1e-10), 
                                 unique_upper_word_count, unique_upper_word_count / (upper_word_count + 1e-10))

        return dataset.withColumn(self.getOutputCol(), 
                                  F.udf(f, VectorUDT())(dataset[self.getInputCol()]))

CPU times: user 166 µs, sys: 31 µs, total: 197 µs
Wall time: 163 µs


In [7]:
%%time
# show model prediction performance on the given dataset
def eval_model_perf(fitted_model, dataset, label_col="label", prediction_col="prediction", probability_col="probability"):
    pred_dataset = fitted_model.transform(dataset)
    eval_dataset = pred_dataset.select(label_col, prediction_col, probability_col)
    # model performance evaluation
    metricNames = ["accuracy", "f1"]
    model_eval = MulticlassClassificationEvaluator(predictionCol=prediction_col, labelCol=label_col)
    for m in metricNames:
        val = model_eval.evaluate(eval_dataset, {model_eval.metricName: m})
        print(m, " = ", val)
    roc_eval = BinaryClassificationEvaluator(rawPredictionCol=probability_col, labelCol=label_col, metricName="areaUnderROC")
    print("AUC =", roc_eval.evaluate(eval_dataset))    
    return pred_dataset

# show CV param tunning result
def show_cv_results(cv_model):
    for result, param in sorted(zip(cv_model.avgMetrics, cv_model.getEstimatorParamMaps()), reverse=True, key=lambda x: x[0]):
        print(result, " | ", param)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


In [8]:
%%time
def run_models(df_train, df_test):
    print("**********LogisticRegression**********")
    t = time.time()
    lr_model = LogisticRegression(featuresCol='features', 
                                  labelCol='label', 
                                  predictionCol='prediction', 
                                  probabilityCol='probability', 
                                  rawPredictionCol='rawPrediction',
                                  family='binomial', 
                                  fitIntercept=True, 
                                  threshold=0.5, 
                                  standardization=False, 
                                  maxIter=200, 
                                  regParam=0.005, 
                                  elasticNetParam=0, 
                                  tol=1e-06, 
                                  aggregationDepth=2)

    lr_model = lr_model.fit(df_train)
    
    eval_model_perf(lr_model, df_test)
    
    print("time taken for LogisticRegression: ", time.time() - t)
    t = time.time()

    # BELOW CODE IS USED ONLY FOR PARAM TUNNING
    # # grid search params tunning
    # paramGrid = ParamGridBuilder() \
    #     .addGrid(lr_model.regParam, [0.001, 0.005]) \
    #     .addGrid(lr_model.standardization, [False, True]) \
    #     .build()
    # # cross validator model
    # crossval = CrossValidator(estimator=lr_model, 
    # #                           evaluator=MulticlassClassificationEvaluator(metricName="accuracy"),
    #                           evaluator=BinaryClassificationEvaluator(rawPredictionCol="probability", labelCol="label", metricName="areaUnderROC"),
    #                           estimatorParamMaps=paramGrid,
    #                           numFolds=2, parallelism = 2,
    #                           seed=666)
    # # run it!
    # crossval_model = crossval.fit(df_train)
    # # evaluate performance
    # show_cv_results(crossval_model)
    # eval_model_perf(crossval_model, df_test)
    # selected_model = crossval_model.bestModel
    # print(selected_model.explainParams())
    print("**********DecisionTreeClassifier**********")
    dt_model = DecisionTreeClassifier(featuresCol='features', 
                                      labelCol='label', 
                                      predictionCol='prediction', 
                                      probabilityCol='probability', 
                                      rawPredictionCol='rawPrediction', 
                                      maxDepth=10, maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0, 
                                      maxMemoryInMB=2048, 
                                      cacheNodeIds=True, 
                                      checkpointInterval=10,
                                      impurity='gini', 
                                      seed=666)

    dt_model = dt_model.fit(df_train)
    eval_model_perf(dt_model, df_test)
    print("time taken for DecisionTreeClassifier: ", time.time() - t)
    t = time.time()
    
    print("**********RandomForestClassifier**********")
    rf_model = RandomForestClassifier(featuresCol='features', 
                                      labelCol='label', 
                                      predictionCol='prediction', 
                                      probabilityCol='probability', 
                                      rawPredictionCol='rawPrediction',
                                      maxDepth=10, 
                                      maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0, 
                                      maxMemoryInMB=2048, 
                                      cacheNodeIds=True, 
                                      checkpointInterval=10, 
                                      impurity='gini', 
                                      numTrees=200, 
                                      featureSubsetStrategy='auto', 
                                      seed=666, 
                                      subsamplingRate=0.8)

    rf_model = rf_model.fit(df_train)
    eval_model_perf(rf_model, df_test)
    print("time taken for RandomForestClassifier: ", time.time() - t)
    t = time.time()
    
    print("**********GBTClassifier**********")
    gbt_model = GBTClassifier(featuresCol='features', 
                             labelCol='label', 
                             maxIter=250)

    gbt_model = gbt_model.fit(df_train)
    eval_model_perf(gbt_model, df_test)
    print("time taken for GBTClassifier: ", time.time() - t)
    t = time.time()    
    
    print("**********MultilayerPerceptronClassifier**********")
    mp_model = MultilayerPerceptronClassifier(featuresCol='features', 
                                              labelCol='label', 
                                              predictionCol='prediction', 
                                              layers=[4, 5, 4, 3],  
                                              maxIter=100, 
                                              blockSize=128, 
                                              seed=1234)

    mp_model = mp_model.fit(df_train)
    eval_model_perf(rf_model, df_test)
    print("time taken for MultilayerPerceptronClassifier: ", time.time() - t)
    t = time.time()  

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.87 µs


In [12]:
%%time
def build_data_preproc_model_with_pca(vocab_size=5000):
    preproc_steps = [
        RegexTokenizer(inputCol="content", outputCol="all_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_words", outputCol="words"),
        CountVectorizer(inputCol="words", outputCol="tf_features", vocabSize=vocab_size),
        IDF(inputCol="tf_features", outputCol="tfidf_features"),
        PCA(inputCol="tfidf_features", outputCol="pca_features", k=100),
        ReviewContentTransformer(inputCol="content", outputCol="content_features"),
        ReviewWordsTransformer(inputCol="words", outputCol="word_features"),
        VectorAssembler(inputCols=["pca_features", "content_features", "word_features"], 
                        outputCol="features")
    ]
    return Pipeline(stages=preproc_steps)

def build_data_preproc_model_without_pca(vocab_size=5000):
    preproc_steps = [
        RegexTokenizer(inputCol="content", outputCol="all_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_words", outputCol="words"),
        CountVectorizer(inputCol="words", outputCol="tf_features", vocabSize=vocab_size),
        IDF(inputCol="tf_features", outputCol="tfidf_features"),
        ReviewContentTransformer(inputCol="content", outputCol="content_features"),
        ReviewWordsTransformer(inputCol="words", outputCol="word_features"),
        VectorAssembler(inputCols=["content_features", "word_features"], outputCol="features")
    ]
    return Pipeline(stages=preproc_steps)

CPU times: user 25 µs, sys: 6 µs, total: 31 µs
Wall time: 26 µs


In [ ]:
%%time
print("**********Run Models with PCA Features**********")
# generate the features to be used for model training
preproc_model = build_data_preproc_model_with_pca(2000).fit(df_train)
df_train_pca = preproc_model.transform(df_train).select("label", "features")
print(df_train_pca.take(1))
df_test_pca = preproc_model.transform(df_test).select("label", "features")
print(df_test_pca.take(1))
run_models(df_train_pca, df_test_pca)

**********Run Models with PCA Features**********
[Row(label=1, features=DenseVector([-2.1915, 6.3671, 5.9627, -0.8709, 1.7528, 0.2703, -2.211, 0.1734, -1.3007, 2.6581, -1.0847, -0.4082, -0.4743, 2.7807, -1.449, 0.0731, -1.0452, -1.0414, -0.047, -0.1921, 0.0957, -1.3617, -0.6802, -1.5515, 1.0233, -0.8923, -1.9182, 0.0896, -0.2538, -0.9743, -0.0991, -0.4519, -0.0602, 0.0667, -0.1877, -0.2286, -0.6194, -0.144, -1.5133, -0.9174, 2.2229, 0.2836, -0.4546, 0.5312, -0.5634, -1.3333, -2.0026, -0.2667, -0.2825, -0.1634, 1.0971, -0.9532, -0.2814, 0.3173, 0.6294, 0.9068, 0.823, 0.4273, 1.9517, -0.3337, -0.4679, 0.6075, 0.6886, -0.8593, 1.4042, 0.4806, -1.6286, 0.0083, 1.3086, -1.5275, -0.8785, -0.8087, 0.653, 0.7002, 0.7796, 0.6142, 0.4945, 0.2033, 0.187, -0.1564, 0.3701, 0.4724, -0.0621, 1.0945, -0.204, -0.1006, -1.3995, -1.1552, 0.3147, 0.551, -0.1518, 0.0239, 0.576, -2.3285, 0.7232, -1.1511, -0.8845, 1.2424, -0.192, 0.9256, 3298.0, 2585.0, 59.0, 0.0228, 273.0, 249.0, 0.9121, 0.0, 0.0, 0.0, 0.0]

In [14]:
%%time
print("**********Run Models without PCA Features**********")
# generate the features to be used for model training
preproc_model = build_data_preproc_model_without_pca(2000).fit(df_train)
df_train_wo_pca = preproc_model.transform(df_train).select("label", "features")
print(df_train_wo_pca.take(1))
df_test_wo_pca = preproc_model.transform(df_test).select("label", "features")
print(df_test_wo_pca.take(1))

nb_model = NaiveBayes(featuresCol='features', 
                      labelCol='label', 
                      predictionCol='prediction', 
                      probabilityCol='probability', 
                      rawPredictionCol='rawPrediction', 
                      smoothing=1, 
                      modelType='multinomial')

nb_model = nb_model.fit(df_train_wo_pca)
eval_model_perf(nb_model, df_test_wo_pca)

**********Run Models without PCA Features**********
[Row(label=1, features=DenseVector([2590.0, 2018.0, 99.0, 0.0491, 230.0, 142.0, 0.6174, 0.0, 0.0, 0.0, 0.0]))]
[Row(label=1, features=DenseVector([6372.0, 5076.0, 125.0, 0.0246, 565.0, 360.0, 0.6372, 0.0, 0.0, 0.0, 0.0]))]
('accuracy', ' = ', 0.5829818903497891)
('f1', ' = ', 0.5803263922596662)
('AUC =', 0.7026301271347141)
CPU times: user 206 ms, sys: 79.1 ms, total: 285 ms
Wall time: 4min 5s
